In [3]:
cd("C:/dev/julia/JuliaStudy")
pwd()

# Loading Datasets
# CSV files
data = readcsv(".\\Data files-selected\\Magic\\magic04.csv")

# # Text files
# f = open(filename, "r")
# lines = readlines(f)
# close(f)

# f = open(filename, "r")
# for line in eachline(f)
#   [some code]

# end
# close(f)

19020×11 Array{Any,2}:
  28.7967   16.0021  2.6449  0.3918  …   -8.2027  40.092    81.8828  "g"
  31.6036   11.7235  2.5185  0.5303      -9.9574   6.3609  205.261   "g"
 162.052   136.031   4.0612  0.0374     -45.216   76.96    256.788   "g"
  23.8172    9.5728  2.3385  0.6147      -7.1513  10.449   116.737   "g"
  75.1362   30.9205  3.1611  0.3168      21.8393   4.648   356.462   "g"
  51.624    21.1502  2.9085  0.242   …    9.8145   3.613   238.098   "g"
  48.2468   17.3565  3.0332  0.2529      10.5868   4.792   219.087   "g"
  26.7897   13.7595  2.5521  0.4236      -2.9292   0.812   237.134   "g"
  96.2327   46.5165  4.154   0.0779      43.1844   4.854   248.226   "g"
  46.7619   15.1993  2.5786  0.3377      -6.6812   7.875   102.251   "g"
  62.7766   29.9104  3.3331  0.2475  …   23.771    9.9144  323.094   "g"
  18.8562   16.46    2.4385  0.5282     -16.9327  11.461   162.848   "g"
  45.6321   22.71    3.0441  0.2213     -14.3164   0.3822  178.255   "g"
   ⋮                        

In [4]:
# p.32 Coding and testing a simple machine learning algorithm in Julia
# Algorithm implemetation
# Listing 2.2 An auxilirary function for the impementation of the kNN algorithm.
# This one is responsible for calculating the distance between two points, x and y, represented as vectors.
function distance{T<:Number}(x::Array{T, 1}, y::Array{T, 1})
  dist = 0              # A
  for i in 1:length(x)  # B
    dist += (x[i] - y[i])^2
  end
  dist = sqrt(dist)
  return dist
end
# A: initializedistance variable
# B: repeat for all dimensions of x and y

distance (generic function with 1 method)

In [5]:
using Base.Test

In [6]:
x = [1, 2, 3]
y = [2, 3, 4]
distance(x, y)

1.7320508075688772

In [6]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`.

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.


In [7]:
isapprox(distance(x, y), 1.7320508075688772)

true

In [11]:
@test distance(x, y) ≈ 1.7320508075688772    

Test Passed
  Expression: distance(x,y) ≈ 1.7320508075688772
   Evaluated: 1.7320508075688772 isapprox 1.7320508075688772

In [7]:
# Listing 2.3 Another auxiliary function of the implementation of the kNN algorithm
# This one performs classification of a point based on tis distances from the known points of the dataset.
function classify{T<:Any}(distances::Array{Float64, 1}, labels::Array{T, 1}, k::Int)
        class = unique(labels)                  # A
        nc = length(class)                      # B
        indexes = Array(Int, k)                 # C
        M = typemax(typeof(distances[1]))       # D
        class_count = Array(Int, nc)
        for i in 1:k
                indexes[i] = indmin(distances)
                distances[indexes[i]] = M       # E
        end
        klabels = labels[indexes]

        for i in 1:nc
                for j in 1:k
                        if klabels[j] == class[i]
                                class_count[i] += 1
                                break
                        end
                end
        end
        index = indmax(class_count)
        return class[index]
end
# A: find all the distinct classes
# B: number of classes
# C: initialize vector of indexes of the nearest neighbors
# D: the largest possible number that this vector can have
# E: make sure this element is not selected again

classify (generic function with 1 method)

In [8]:
N = size(X, 1)                          # A
n = size(Y, 1)                          # B
D = Array(Float64, N)                   # C
z = Array(typeof(x[1]), n)              # D
for i in 1:n
                for j in 1:N
                        D[j] = distance(X[j, :], Y[i, :])
                end
end

LoadError: UndefVarError: X not defined

In [41]:
D

4-element Array{Float64,1}:
 268.359
 248.875
 239.59 
 204.249

In [51]:
@test classify(D, x, 1) == "g"
@test classify(D, x, 2) == "g"
@test classify(D, x, 3) == "g"
@test classify(D, x, 4) == "h"

Test Passed
  Expression: classify(D,x,4) == "h"
   Evaluated: "h" == "h"

In [90]:
# Listing 2.4 The main function(wrapper) of the implementation of the kNN algorithm.
function apply_kNN{T1<:Number, T2<:Any}(X::Array{T1, 2}, x::Array{T2, 1}, Y::Array{T1, 2}, k::Int)
        N = size(X, 1)                          # A
        n = size(Y, 1)                          # B
        D = Array(Float64, N)                   # C
        z = Array(typeof(x[1]), n)              # D
        for i in 1:n
                for j in 1:N
                        D[j] = distance(X[j, :], Y[i, :])
                end
                z[i] = classify(D, x, k)
        end
        return z
end

# A: number of known data points
# B: number of data points to classify
# C: initialize distance vector
# D: initialize labels vector (output)

apply_kNN (generic function with 1 method)

apply_kNN(Array{#T1<:Number, 2}, Array{#T2<:Any, 1}, Array{#T1<:Number, 2}, Int64) in module Main at In[30]:3 overwritten at In[90]:3.


In [87]:
@test string.(apply_kNN(X, x, Y, 1)) == ["h", "g", "g", "g"]
@test string.(apply_kNN(X, x, Y, 2)) == ["g", "g", "g", "g"]
@test string.(apply_kNN(X, x, Y, 3)) == ["h", "h", "g", "h"]
@test string.(apply_kNN(X, x, Y, 4)) == ["g", "g", "g", "h"]

Test Failed
  Expression: string.(apply_kNN(X,x,Y,1)) == ["h","g","g","g"]
   Evaluated: SubString{String}["h","g","h","g"] == String["h","g","g","g"]


LoadError: LoadError: There was an error during testing
while loading In[87], in expression starting on line 1

In [26]:
data_g = magic_data[1:5, :]
data_h = magic_data[end-2:end, :]

data = vcat(data_g, data_h)

3×11 Array{Any,2}:
  75.4455  47.5305  3.4483  0.1417  …   -9.4662  30.2987  256.517  "h"
 120.513   76.9018  3.9939  0.0944     -63.8389  84.6874  408.317  "h"
 187.181   53.0014  3.2093  0.2876      31.4755  52.731   272.317  "h"

In [100]:
data

8×11 Array{Any,2}:
  28.7967   16.0021  2.6449  0.3918  …   -8.2027  40.092    81.8828  "g"
  31.6036   11.7235  2.5185  0.5303      -9.9574   6.3609  205.261   "g"
 162.052   136.031   4.0612  0.0374     -45.216   76.96    256.788   "g"
  23.8172    9.5728  2.3385  0.6147      -7.1513  10.449   116.737   "g"
  75.1362   30.9205  3.1611  0.3168      21.8393   4.648   356.462   "g"
  75.4455   47.5305  3.4483  0.1417  …   -9.4662  30.2987  256.517   "h"
 120.513    76.9018  3.9939  0.0944     -63.8389  84.6874  408.317   "h"
 187.181    53.0014  3.2093  0.2876      31.4755  52.731   272.317   "h"

In [111]:
# p.38 Algorithm testing
# data = readcsv("magic04.txt")

I = map(Float64, data[:, 1:(end - 1)])  # A
O = data[:, end]                        # B

# A: take all the columns of the data matrix, aprot from the last one andconvert
#    everything into a Float. Result = 10-dimArray of Float numbers
# B: take only the lastcolumnof the data matrix.
#    Result = 1-dim Array

# Listing 2.5 Code for testingthe implementation of the kNN algorithm,
# using the preloaded Magic dataset.
N = length(O)                   # A
n = round(Int64, N/2)           # B
# R = randperm(N)
R = [4, 5, 2, 7, 6, 1, 8, 3]    # C
indX = R[1:n]                   # D
X = I[indX, :]                  # E
x = O[indX]                     # F
indY = R[(n+1):end]             # G
Y = I[indY, :]                  # E
y = O[indY]
# A: number of data pints in the whole dataset(which is equivalent to the lenght of array O)
# B: the half of the above number
# C: a random permutation of all the indexes(essential for sampling)
# D: get some random indexes for the training set
# E: input values for training and testing set respectively
# F: target values for training and testing set respectively
# G: some random indexes for the testing set

z = apply_kNN(X, x, Y, 5)

4-element Array{SubString{String},1}:
 "h"
 "h"
 "h"
 "h"

In [109]:
X, x, Y

(
[23.8172 9.5728 … 10.449 116.737; 75.1362 30.9205 … 4.648 356.462; 31.6036 11.7235 … 6.3609 205.261; 120.513 76.9018 … 84.6874 408.317],

Any["g","g","g","h"],
[75.4455 47.5305 … 30.2987 256.517; 28.7967 16.0021 … 40.092 81.8828; 187.181 53.0014 … 52.731 272.317; 162.052 136.031 … 76.96 256.788])

gg


In [ ]:
# p.39 Saving your workspace into a data file
# Saving data into delimited files
writedlm("/data/mydata.dat", A, ";")

writecsv("/data/mydata.dat", A)

# Saving data into Native Julia format; other libraries are needed
# Pkg.add("HDF5")
# Pkg.add("JLD")
using JLD
f = open("mydata.jld", "w")
@write f A      # write variable A into a file f
@write f b      # write variable b into a file f
close(f)
# Alternative way of writing into .jld fformat
save("mydata.jld", "var_A", A, "var_B", b)
# if you wish to save all the variables in the workspace,
save("mydata.jld")
# to retrive the data stored in a .jld file,
D = load("mydata.jld")
# to access only a particular variable in a .jld file
b = load("mydata.jld", "var_b")
# to access the variables stored in a .jld file partially
f = jldopen("mydata.jld", "r")
dump(f, 20)     # reading the first 20 variables from the file

In [ ]:
# Saving data into text files
f = open("/data/mydata.txt", "w")
write(f, SomeStringVariable)
write(f, AnotherStringVariable)
# ...
close(f)
# to save a string data to a data file which is already open
A = [123, 34423.23, -322, 981238651928736918263]
f = open("p43example.txt", "w")
for a in A
        write(f, string(a, "\n"))
end
close(f)